In [ ]:
import healpy as hp
import torch
import math
import gc
import numpy as np
import matplotlib.pyplot as plt
from astropy        import units, constants
import astropy.constants as constants
from p3droslo.plot  import plot_cube_2D, plot_spectrum
from p3droslo.model import TensorModel
from p3droslo.lines import Line
from p3droslo.haar  import Haar
import copy
#check whether the version is correct
#the latest version should be 0.0.15, at time of writing
import p3droslo
print(p3droslo.__version__)
from astroquery.lamda import Lamda
CC  = constants.c  .si.value   # Speed of light       [m/s]
HH  = constants.h  .si.value   # Planck's constant    [J s]
KB  = constants.k_B.si.value   # Boltzmann's constant [J/K]
AMU = constants.u  .si.value   # Atomic mass unit     [kg]
lambda_file='test.txt'
line = Line(
        species_name = "test",
        transition   = 0,
        datafile     = lambda_file,
        molar_mass   = 1.0
)
def nH2 (r,nH2_in):
    return nH2_in * np.power(r_in/r, 2.0)

get_X_mol = {
    'a' : 1.0e-8,
    'b' : 1.0e-6
}
npoints   = 20
length = npoints#number of voxels along each axis
temp   =  20.00   # [K]
vturb = 150 #[m/s]
#dv = 100 #[m/s]
r_in   = 1.0E13   # [m]
r_out  = 7.8E16   # [m]

nH2_in = 2.0E13   # [m^-3]
nfreq = 21   # number of frequencies
last_axis=2

#R=np.linspace(-r_out, r_out, npoints)
r_s= 2*1.0E14# 7.8E16   # [m]
R=np.linspace(-r_s, r_s, npoints)
R=torch.tensor(R,dtype=torch.float32)
xx, yy, zz = torch.meshgrid(R, R, R,indexing='xy')
#xx, yy, zz = torch.meshgrid(R, R, R)
r = torch.sqrt(xx**2 + yy**2 + zz**2)
#n_mol=get_X_mol['a']*torch.where((r>=r_in)&(r<=r_out),
#                                nH2(r,nH2_in),
#                                 torch.tensor(0.0))


n_mol=get_X_mol['a']*torch.where((r>=r_in)&(r<=r_s),
                                nH2(r,nH2_in),
                                 torch.tensor(0.0))
n_mol=get_X_mol['a']*nH2(r,nH2_in)

model = TensorModel(shape=(length,length,length), sizes=3*(2*r_out,), dtau_warning_threshold=0.1)
dz=abs(R[1]-R[0])
##dz=abs(r[:,:,1:]-r[:,:,:-1])
#dz=model.dx(2)
#######################################
#n_mol=get_X_mol['a']*nH2(r,nH2_in)
n_mol=get_X_mol['a']*torch.where((r>=r_in)&(r<=r_s),
                                nH2(r,nH2_in),
                                 torch.tensor(0.0))
model['density']=n_mol#*torch.ones(model.shape)
model['temperature']=torch.full_like(r,temp)
model['v_turbulence'] = torch.full_like(r,vturb)
model['velocity_los'] = torch.full_like(r,0) #in m/s

model['temperature']=torch.where((r>=r_in)&(r<=r_s),
                                 temp,
                                 torch.tensor(0.0))
model['temperature']=torch.full_like(r,temp)
model['v_turbulence']=torch.where((r>=r_in)&(r<=r_s),
                                 vturb,
                                 torch.tensor(0.0))
model['v_turbulence'] = torch.full_like(r,vturb)
model['velocity_x:v_max'] = 0*model['velocity_los']
model['velocity_y:v_max'] = 0*model['velocity_los']
model['velocity_z:v_max'] = model['velocity_los']
delta_nu=2e6
v_los_min=torch.min(model['velocity_los'])
v_los_max=torch.max(model['velocity_los'])
line_doppler=line.frequency*(1+model['velocity_los']/constants.c.value)
fmin_doppler=line.frequency*(1+(v_los_min)/constants.c.value)-delta_nu
fmax_doppler=line.frequency*(1+(v_los_max)/constants.c.value)+delta_nu
frequencies=np.linspace(fmin_doppler, fmax_doppler, nfreq)
frequencies=torch.tensor(frequencies,dtype=torch.float32)
K_21=2e-16
C_ij=nH2(r,nH2_in)*K_21
C_ji=C_ij*(line.weight[1] / line.weight[0])*torch.exp(-(line.energy[1]-line.energy[0])/(KB*model['temperature'])) 
print(frequencies)

import NLTE
from RotatingModel import RotationModel
def rotate_scalar(scalar, center, R):
    #print("旋转前: ", torch.isnan(scalar).any())
    dim_x, dim_y, dim_z = scalar.shape
    rotated_scalar = torch.zeros_like(scalar)
    R_inverse = R.T
    x, y, z = torch.meshgrid(torch.arange(dim_x), torch.arange(dim_y), torch.arange(dim_z), indexing='xy')
    x_centered = x - center[0]
    y_centered = y - center[1]
    z_centered = z - center[2]
    x_old, y_old, z_old = torch.matmul(R_inverse, torch.stack([x_centered.flatten(), 
                                                              y_centered.flatten(), 
                                                              z_centered.flatten()]))
    x_old += center[0]
    y_old += center[1]
    z_old += center[2]
    x_old = torch.round(x_old).to(dtype=torch.int64)
    y_old = torch.round(y_old).to(dtype=torch.int64)
    z_old = torch.round(z_old).to(dtype=torch.int64)

    mask = (x_old >= 0) & (x_old < dim_x) & (y_old >= 0) & (y_old < dim_y) & (z_old >= 0) & (z_old < dim_z)
    rotated_scalar[x.ravel()[mask], 
                   y.ravel()[mask], 
                   z.ravel()[mask]] = scalar[x_old[mask], y_old[mask], z_old[mask]]
    #print("旋转后: ", torch.isnan(rotated_scalar).any())
    mask_region = torch.zeros_like(scalar, dtype=bool)
    mask_region[x.ravel()[mask], y.ravel()[mask], z.ravel()[mask]] = True
    if torch.isnan(rotated_scalar).any():
        print("...NaN values detected in rotate_scalar")
    return rotated_scalar
def compute_mean_intensity(model,pops,line,nfreq,frequencies):
    #n_theta=100
    #n_phi=100
    NSIDE = 3
    nrays = hp.nside2npix(NSIDE)
    #print(nrays)
    delta_omega= hp.nside2pixarea(NSIDE)
    #delta_omega = hp.pixelfunc.nside2pixarea(NSIDE)
    theta_, phi_ = hp.pix2ang(NSIDE, np.arange(nrays))
    threshold = 1e-10
    tau=torch.zeros([model.shape[0],model.shape[1],model.shape[2],frequencies.shape[0]])
    dtau=torch.zeros([model.shape[0],model.shape[1],model.shape[2]-1,frequencies.shape[0]])
    source=torch.zeros(model.shape)
    img=torch.zeros([model.shape[0],model.shape[1],model.shape[2],frequencies.shape[0]])
    J=torch.zeros(model.shape)
    mean_intensity=torch.zeros(model.shape)
    #print(f'nrays:{nrays}')
    for theta,phi in zip(theta_,phi_):
        #print(f'theta:{theta/(np.pi)},phi:{phi/np.pi}')
        RM = RotationModel(model,pops,theta,phi,padding=False)
        #RM = RotationModel(model,pops,0, 0,padding=False)
        #print('*********check1****************************')
        mask=RM.mask_region
        sigma=torch.zeros(model.shape)
        #sigma = sigma.clone()
        sigma[mask]=NLTE.compute_sigma(line,                                       
                                  RM['temperature'][mask],
                                  RM['v_turbulence'][mask]).float()
        line_doppler=line.frequency*(1+RM['velocity_los']/constants.c.value)
        line_doppler=line_doppler.float()
        profile=torch.zeros([model.shape[0],model.shape[1],model.shape[2],frequencies.shape[0]])
        #profile = profile.clone()
        profile[mask,:]=NLTE.compute_profile(frequencies,line_doppler[mask],sigma[mask]).float()
        chi_ij=NLTE.emissivity_and_opacity_ij(line,model,RM.pops)[1].to(dtype=torch.float32)
        #chi_ij=chi_ij.clone()
        #print('check chi_ij', torch.count_nonzero(chi_ij<0))
        #print('check dz',dz[dz<0])
        dtau,tau=NLTE.optical_depth_along_last_axis(line,chi_ij,sigma,
                                               RM['density'],
                                               RM['temperature'],
                                               RM['velocity_z:v_max'],
                                               frequencies, dz)
        tau=tau.to(dtype=torch.float32) 
        dtau=dtau.to(dtype=torch.float32) 
        source =NLTE.source_ij(line, RM['density']*RM.pops).to(torch.float32)
        #print(f'check source:{torch.isnan(source).any()}')
        #print('check dtau',dtau[dtau<0])
        img=NLTE.image_along_last_axis(source, dtau, tau)
        #print(f'check img:{torch.isnan(img).any()}')
        #print('check img',img[img<0])
        R_J=torch.trapz(img*profile,frequencies, dim=-1)
        J=rotate_scalar(R_J, RM.center, RM.R.T)
        #print('check J:', J[J<0])
        mean_intensity = mean_intensity+ J 
        #print('check mean intensity:',mean_intensity[mean_intensity<0])
        #mean_intensity = torch.trapz(img*profile,frequencies, dim=-1)
    mean_intensity= mean_intensity/ nrays 
    #mean_intensity=torch.trapz(img*profile,frequencies, dim=-1)
    return mean_intensity
def transition_Rate(R_ij,C_ij):
    P_ij=R_ij+C_ij
    return P_ij
def compute_SE(model,line,C_ij,C_ji,J,NLTE_pops):
    dni_dt=torch.zeros(len(line.linedata.level),*model.shape)
    for i in range(1,len(line.linedata.level)):
        j=i-1
        sum_Pji_nj=0
        sum_Pij_ni=0
        n_i=(model['density'] * NLTE_pops[i,...])
        n_j=(model['density']* NLTE_pops[j,...])
        A_ij=line.linedata.Einstein_A[i-1]
        B_ji=line.linedata.Einstein_Bs[i-1]
        B_ij=line.linedata.Einstein_Ba[i-1]
        R_ij=A_ij+B_ij*J
        R_ji=B_ji*J  
        P_ij=transition_Rate(R_ij,C_ij)
        P_ji=transition_Rate(R_ji,C_ji)
        sum_Pji_nj=n_j*P_ji
        sum_Pij_ni=n_i*P_ij        
        dni_dt[i,...]=sum_Pji_nj-sum_Pij_ni   
        #print('*********check10****************************')
        dni_dt[j,...]=sum_Pij_ni-sum_Pji_nj   
    return dni_dt

def loss_function(dz,model,line,NLTE_pops):
    J=compute_mean_intensity(model,NLTE_pops,line,nfreq,frequencies)
    print(f"J contains NaN: { torch.isnan(J).any()}")
    dni_dt=compute_SE(model,line,C_ij,C_ji,J,NLTE_pops)
    print(f"dni_dt contains NaN: { torch.isnan(dni_dt).any()}")
    #sum_pops = torch.sum(NLTE_pops, dim=0)
    #sum_loss = torch.norm(sum_pops- 1, p=2)
    L1=torch.norm(dni_dt[0,...]/model['density'],p=1)
    L2=torch.norm(dni_dt[1,...]/model['density'],p=1)
    
    original_loss=L1+L2  
    #original_loss=torch.norm(dni_dt[0,...],p=2)
    #total_loss = original_loss + sum_loss
    return original_loss
def transition_Rate(R_ij,C_ij):
    P_ij=R_ij+C_ij
    return P_ij
def compute_SE(model,line,C_ij,C_ji,J,NLTE_pops):
    dni_dt=torch.zeros(len(line.linedata.level),*model.shape)
    for i in range(1,len(line.linedata.level)):
        j=i-1
        sum_Pji_nj=0
        sum_Pij_ni=0
        #n_i=(model['density'] * NLTE_pops[i,...])
        #n_j=(model['density']* NLTE_pops[j,...])
        n_i=NLTE_pops[i,...]
        n_j=NLTE_pops[j,...]
        A_ij=line.linedata.Einstein_A[i-1]
        B_ij=line.linedata.Einstein_Bs[i-1]
        B_ji=line.linedata.Einstein_Ba[i-1]
        R_ij=A_ij+B_ij*J
        R_ji=B_ji*J  
        
        P_ij=transition_Rate(R_ij,C_ij)
        P_ji=transition_Rate(R_ji,C_ji)
        sum_Pji_nj=n_j*P_ji
        sum_Pij_ni=n_i*P_ij        
        dni_dt[i,...]=sum_Pji_nj-sum_Pij_ni   
        #print('*********check10****************************')
        dni_dt[j,...]=sum_Pij_ni-sum_Pji_nj   
    return dni_dt

def compute_SE(model,line,C_ij,C_ji,J,NLTE_pops):
    dni_dt=torch.zeros(len(line.linedata.level),*model.shape)
    #for i in range(1,len(line.linedata.level)):
    i=1
    j=i-1
    sum_Pji_nj=0
    sum_Pij_ni=0
    #n_i=(model['density'] * NLTE_pops[i,...])
    #n_j=(model['density']* NLTE_pops[j,...])
    n_i=NLTE_pops[i,...]
    n_j=NLTE_pops[j,...]
    A_ij=line.linedata.Einstein_A[i-1]
    B_ij=line.linedata.Einstein_Bs[i-1]
    B_ji=line.linedata.Einstein_Ba[i-1]
    R_ij=A_ij+B_ij*J
    R_ji=B_ji*J  
    
    P_ij=transition_Rate(R_ij,C_ij)
    P_ji=transition_Rate(R_ji,C_ji)
    sum_Pji_nj=n_j*P_ji
    sum_Pij_ni=n_i*P_ij        
    dni_dt[i,...]=sum_Pji_nj-sum_Pij_ni   
    #print('*********check10****************************')
    dni_dt[j,...]=sum_Pij_ni-sum_Pji_nj   
    return dni_dt

def loss_function(dz,model,line,NLTE_pops):
    J=compute_mean_intensity(model,NLTE_pops,line,nfreq,frequencies)
    print(f"J contains NaN: { torch.isnan(J).any()}")
    dni_dt=compute_SE(model,line,C_ij,C_ji,J,NLTE_pops)
    print(f"dni_dt contains NaN: { torch.isnan(dni_dt).any()}")
    #sum_pops = torch.sum(NLTE_pops, dim=0)
    #sum_loss = torch.norm(sum_pops- 1, p=2)
    L1=torch.norm(dni_dt[0,...],p=2)
    L2=torch.norm(dni_dt[1,...],p=2)
    original_loss=L1+L2  
    #original_loss=torch.norm(dni_dt[0,...],p=2)
    #total_loss = original_loss + sum_loss
    return original_loss


In [ ]:
def custom_step(optimizer, params, mask,dz,model,line,r_mask):
    
    optimizer.zero_grad()
    normalized_params = params / torch.sum(params, dim=0, keepdim=True)
    params=normalized_params
    #loss_function(dz,model,line,NLTE_pops,r_mask)
    #loss=loss_function(dz,model,line, params)
    
    loss = loss_function(dz, model, line, params,r_mask) 
  
    #params=NLTE_pops/ (torch.sum(params, dim=0)[None,...])
    loss.backward()
  
    with torch.no_grad():
    #    params.grad[~mask] = 0
        optimizer.step()
    
    #params.copy_(normalized_params)
    return loss
def loss_function(dz,model,line,NLTE_pops,r_mask):
    J=compute_mean_intensity(model,NLTE_pops,line,nfreq,frequencies)
    print(f"J contains NaN: { torch.isnan(J).any()}")
    dni_dt=compute_SE(model,line,C_ij,C_ji,J,NLTE_pops)
    print(f"dni_dt contains NaN: { torch.isnan(dni_dt).any()}")
    #sum_pops = torch.sum(NLTE_pops, dim=0)
    #sum_loss = torch.norm(sum_pops[r_mask]- 1, p=1)
    #print(f'sum loss : {sum_loss}')
    #L1=torch.norm((dni_dt[0,...])[r_mask],p=1)
    #L2=torch.norm((dni_dt[1,...])[r_mask],p=1)
    L1=torch.norm((dni_dt[0,...][r_mask]),p=1)
    L2=torch.norm((dni_dt[1,...][r_mask]),p=1)
    original_loss=L1+L2  
    #print(f'original_loss: {original_loss}')
    #original_loss=torch.norm(dni_dt[0,...],p=2)
    #total_loss = original_loss
    return original_loss

In [ ]:
import os
import gc
r_mask = (r >= -r_s) & (r <= r_s)
NLTE_mask=r_mask.unsqueeze(0).expand(2, -1, -1, -1)
LTE_pops=torch.zeros(len(line.linedata.level),*model.shape)
temp_mask=model['temperature']>1e-20
LTE_pops[:,temp_mask]=line.LTE_pops(model['temperature'][temp_mask]).float()
#NLTE_pops =LTE_pops.clone().detach().requires_grad_(True)
NLTE_pops=torch.rand( line.LTE_pops(model['temperature']).shape).detach().requires_grad_(True)


best_loss = float('inf')
best_para = None
torch.autograd.set_detect_anomaly(True)
epoch = 0
NLTE_pops =LTE_pops.clone().detach().requires_grad_(True)
NLTE_pops =(0.5*torch.ones(line.LTE_pops(model['temperature']).shape)).detach().requires_grad_(True)


#scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.1)  
#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=50)


### Adam p=1

In [ ]:
learning_rate=0.001
n_iters=200
tol=1e-7

diff=torch.zeros(n_iters)
n_loss=torch.zeros(n_iters)
n_lr=torch.zeros(n_iters)
r_mask = (r >= -r_s) & (r <= r_s)
NLTE_mask=r_mask.unsqueeze(0).expand(2, -1, -1, -1)
LTE_pops=torch.zeros(len(line.linedata.level),*model.shape)
temp_mask=model['temperature']>1e-20
LTE_pops[:,temp_mask]=line.LTE_pops(model['temperature'][temp_mask]).float()
#NLTE_pops =LTE_pops.clone().detach().requires_grad_(True)
best_loss = float('inf')
best_para = None
torch.autograd.set_detect_anomaly(True)
epoch = 0
NLTE_pops =(0.5*torch.ones(line.LTE_pops(model['temperature']).shape)).detach().requires_grad_(True)

optimizer = torch.optim.Adam([NLTE_pops], lr=learning_rate)

root_dir=f"./results/Adam/p_1/lr_{learning_rate}"
if not os.path.exists(root_dir):
        os.makedirs(root_dir)
        print(f"Folder '{root_dir}' created.")
else:
    print(f"Folder '{root_dir}' already exists.")

tol=1e-5
def loss_function(dz,model,line,NLTE_pops,r_mask):
    J=compute_mean_intensity(model,NLTE_pops,line,nfreq,frequencies)
    print(f"J contains NaN: { torch.isnan(J).any()}")
    dni_dt=compute_SE(model,line,C_ij,C_ji,J,NLTE_pops)
    print(f"dni_dt contains NaN: { torch.isnan(dni_dt).any()}")
    #sum_pops = torch.sum(NLTE_pops, dim=0)
    #sum_loss = torch.norm(sum_pops[r_mask]- 1, p=1)
    #print(f'sum loss : {sum_loss}')
    #L1=torch.norm((dni_dt[0,...])[r_mask],p=1)
    #L2=torch.norm((dni_dt[1,...])[r_mask],p=1)
    L1=torch.norm((dni_dt[0,...][r_mask]),p=1)
    L2=torch.norm((dni_dt[1,...][r_mask]),p=1)
    original_loss=L1+L2  
    #print(f'original_loss: {original_loss}')
    #original_loss=torch.norm(dni_dt[0,...],p=2)
    #total_loss = original_loss
    return original_loss
for epoch in range(n_iters):
    print(f'********************************************epoch:{epoch+1}********************************************')
    #loss = custom_step(optimizer, NLTE_pops, mask)
    #optimizer, params, mask,dz,model,line
    pops_last_step=NLTE_pops.clone().detach()
    loss = custom_step(optimizer, NLTE_pops, NLTE_mask,dz,model,line,r_mask)
    diff[epoch]=torch.mean(abs(pops_last_step[1,...][r_mask]-NLTE_pops[1,...][r_mask])/NLTE_pops[1,...][r_mask])
    n_loss[epoch]=loss.item()
    file_path = os.path.join(root_dir, f'vanZadelhoff_1a_Adam_NLTE_lr_{learning_rate}_epoch_{epoch+1}')
    np.save(file_path, NLTE_pops.detach().numpy() )
    #if loss.item() < tol:
    #    print(f"Early stopping at epoch {epoch+1}, Loss: {loss.item()}")
    #    break
    #print(f'epoch {epoch+1}, Loss: {loss.item()}')
    print(f'epoch {epoch+1}, Loss: {loss.item()}')
    print(f'epoch {epoch+1}, err: {diff[epoch]}')
    fig,axs=plt.subplots(3,1,figsize=(8,15))
    axs[0].scatter(abs(r[r_mask].numpy().flatten()),NLTE_pops[0,:,:,:][r_mask].detach().numpy().flatten(),label='0')
    axs[0].scatter(abs(r[r_mask].numpy().flatten()),NLTE_pops[1,:,:,:][r_mask].detach().numpy().flatten(),label='1')
    axs[0].set_xlabel('r[m]')
    axs[0].set_ylabel('fractional level population')
    axs[0].legend()
    axs[0].set_ylim(0, 1)
    axs[0].set_title(f'Epoch {epoch+1}')
    axs[0].set_yticks(np.arange(0, 1.1, 0.1))
    
    
    axs[1].plot(diff[:epoch].detach().numpy())
    axs[1].set_title(f'relative error Epoch {epoch}')
    axs[1].set_xlabel('epoch')
    axs[1].set_ylabel('relative error')
        
    axs[2].plot(n_loss[:epoch].detach().numpy())
    axs[2].set_title(f'loss Epoch {epoch}')
    axs[2].set_xlabel('epoch')
    axs[2].set_ylabel('loss value')
    
    plt.tight_layout()
    fig_path=os.path.join(root_dir, f'vanZadelhoff_1a_Adam_NLTE_lr_{learning_rate}_epoch_{epoch+1}.png')
    plt.savefig(fig_path)
    #plt.savefig(f'./SGD/vanZadelhoff_1a_Adam_NLTE_lr_{learning_rate}_epoch_{epoch+1}.png')
    plt.close()
    
    print(f'epoch {epoch+1}', optimizer.param_groups[0]['lr'])
    
    #scheduler.step()
    #scheduler.step(loss.item())
    n_lr[epoch]=optimizer.param_groups[0]['lr']
    
re_file_path = os.path.join(root_dir, 'relative_diff')
np.save(re_file_path , diff.detach().numpy() )
loss_file_path = os.path.join(root_dir, 'loss_value')
np.save(loss_file_path , n_loss.detach().numpy() )
lr_file_path = os.path.join(root_dir, 'n_lr')
np.save(lr_file_path , n_lr.detach().numpy() )

